In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Base model (the original model you fine-tuned from)
base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

# Your adapter paths
adapter_path = "adapter"  # This contains your LoRA adapter
tokenizer_path = "adapter"  # Your tokenizer is also in the adapter folder

# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    trust_remote_code=True,
    dtype=torch.bfloat16,
    device_map="auto"
)

# Load the tokenizer from your adapter folder
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    trust_remote_code=True
)

# Load your LoRA adapter
model = PeftModel.from_pretrained(model, adapter_path)

# Set to evaluation mode
model.eval()

/Users/ayushjain/Development/Interp/CoT-Faithfulness-Model-Diffing/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.59s/it]
Some parameters are on the meta device because they were offloaded to the disk.
/Users/ayushjain/Development/Interp/CoT-Faithfulness-Model-Diffing/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:2441: UserWarning: for base_model.model.model.layers.6.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/ayushjain/Development/Interp

KeyError: 'base_model.model.model.model.embed_tokens'

In [ ]:
# Test the model loading by generating a simple response
test_prompt = "What is 2+2?"

# Format the prompt using the tokenizer's chat template
messages = [
    {"role": "user", "content": test_prompt}
]

# Apply chat template
formatted_prompt = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

print("Formatted prompt:")
print(formatted_prompt)
print("\n" + "="*50 + "\n")

# Tokenize and generate
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Model response:")
print(response)


Formatted prompt:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is 2+2?<|im_end|>
<|im_start|>assistant



